# Project Group - 22

Members: Charlotte Koning, Cleo Vos, Koen Awater, Jasmijn van de Kar, Jippo Steenstra

Student numbers: 5414164, 5324599, 5416493, 5315611, 5064740

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

objective: 
To what extent can crowd prediction help to identify and mitigate safety risks, such as overcrowded areas, at an early stage?

questions:
- Which factors contribute most strongly to safety risks during large-scale public events?  
- How accurately can crowd density be forecasted using historical and real-time data?  
- Which prediction methods provide the best performance for short-term crowd prediction?  



**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

We will use the dataset(s) about the SAIL event, provided by the professors.

# Data Pipeline

For the geographical scale we will use eastern harbor of Amsterdam IJhaven, across the IJ river. The temporal scale of the project will be the 20-24 august 2025. For data preperation we will use the Panda library in python.